In [ ]:
# RAG with Nomic + FAISS + Llama 3.1 

import os
import pandas as pd
import numpy as np
import faiss
from tqdm import tqdm
from nomic import embed
from transformers import pipeline
from bs4 import BeautifulSoup

# ------------------ CONFIG ------------------
HF_TOKEN = "hf_token"  # tu token de Hugging Face
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN

MODEL_EMBEDDINGS = "nomic-embed-text-v1.5"
MODEL_LLM = "meta-llama/Llama-3.1-8B-Instruct"

DATA_PATH = "/home/jovyan/lrec_2026/data_10/UN/glossary_UN_info_semantica_es.csv"
OUTPUT_PATH = "/home/jovyan/lrec_2026/data_10/UN/UN_rag_es_llama.csv"

LIMIT_TERMS = 100
TOP_K = 3

# ------------------ CARGA CSV ------------------
print("📥 Cargando CSV...")
df = pd.read_csv(DATA_PATH, sep=";", quotechar='"', engine="python", on_bad_lines="skip")
print(f"✅ CSV cargado con {len(df)} filas y columnas: {list(df.columns)}")

if "term" not in df.columns or "info_semantica" not in df.columns:
    raise ValueError("❌ El CSV debe tener las columnas 'term' y 'info_semantica'.")

# Limitar cantidad de términos (para pruebas)
df = df.head(LIMIT_TERMS)

# ------------------ LIMPIEZA ------------------
def clean_html(text):
    if pd.isna(text):
        return ""
    return BeautifulSoup(str(text), "html.parser").get_text(separator=" ", strip=True)

df["info_semantica"] = df["info_semantica"].apply(clean_html).fillna("")
df["term"] = df["term"].astype(str).fillna("")

# Crear documentos combinando término e info semántica
documents = [
    f"Término: {t}\nContexto semántico: {c}" for t, c in zip(df["term"], df["info_semantica"])
]

print(f"📘 Procesando {len(documents)} documentos para embeddings...")

# ------------------ EMBEDDINGS NOMICS ------------------
def get_nomic_embeddings(texts, model=MODEL_EMBEDDINGS):
    """Devuelve los embeddings (vectores) de una lista de textos"""
    res = embed.text(texts=texts, model=model, task_type="search_document")
    return np.array(res["embeddings"], dtype="float32")

def l2_normalize(a, axis=1, eps=1e-10):
    norms = np.linalg.norm(a, axis=axis, keepdims=True)
    return a / (norms + eps)

print("🔹 Generando embeddings con Nomic...")
embeddings = get_nomic_embeddings(documents)
embeddings = l2_normalize(embeddings)
print(f"✅ Embeddings generados con forma: {embeddings.shape}")

# ------------------ INDEX FAISS ------------------
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)  # inner product ≈ similitud coseno
index.add(embeddings)
print("✅ Índice FAISS creado y poblado")

# ------------------ FUNCIÓN DE RECUPERACIÓN ------------------
def rag_retrieve(term, info_semantica, top_k=TOP_K, include_self=True):
    """
    Recupera los documentos más parecidos usando embeddings.
    include_self=True permite incluir el propio documento.
    """
    query_text = f"Término: {term}\nContexto semántico: {info_semantica}"
    q_emb = get_nomic_embeddings([query_text])
    q_emb = l2_normalize(q_emb)
    D, I = index.search(q_emb, top_k)
    retrieved = [documents[i] for i in I[0]]

    if not include_self:
        retrieved = [d for d in retrieved if not d.startswith(f"Término: {term}\n")]

    return retrieved

# ------------------ MODELO LLM ------------------
print("🔹 Cargando modelo Llama 3.1...")
llm_pipeline = pipeline(
    "text-generation",
    model=MODEL_LLM,
    tokenizer=MODEL_LLM,
    device_map="auto",
    torch_dtype="auto",
    max_new_tokens=256,
    temperature=0.5,
    repetition_penalty=1.1
)
print("✅ Modelo cargado correctamente")

# ------------------ PROMPT ------------------
PROMPT_TEMPLATE = """Eres un experto en terminología.
Basándote únicamente en la siguiente información semántica, redacta una definición precisa y concisa en español para el término: "{term}".
No inventes información; si la información no es suficiente, escribe "No hay información suficiente para definir el término."

Información semántica (contexto recuperado):
{context}

Definición:
"""

# ------------------ GENERAR DEFINICIONES ------------------
def generate_definition(term, info_semantica, top_k=TOP_K):
    retrieved = rag_retrieve(term, info_semantica, top_k=top_k, include_self=True)
    context = "\n\n---\n\n".join(retrieved).strip() or "No hay información semántica disponible."

    prompt = PROMPT_TEMPLATE.format(term=term, context=context)
    output = llm_pipeline(prompt)[0]["generated_text"]

    if "Definición:" in output:
        definition = output.split("Definición:")[-1].strip()
    else:
        definition = output.strip()

    return definition.split("\n")[0].strip()

# ------------------ PROCESAMIENTO ------------------
definitions = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="🧠 Generando definiciones (RAG)"):
    try:
        definition = generate_definition(row["term"], row["info_semantica"], top_k=TOP_K)
    except Exception as e:
        definition = f"ERROR: {e}"
    definitions.append(definition)

df["definition"] = definitions

df.to_csv(OUTPUT_PATH, sep=";", index=False)
print(f"\n✅ Archivo final guardado en: {OUTPUT_PATH}")
print(f"📘 Términos procesados: {len(df)}")
